# motion detection and trackeing

In [2]:
def stackImages(scale, imgArray):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range(0, rows):
            for y in range(0, cols):
                if imgArray[x][y].shape[:2] == imgArray[0][0].shape[:2]:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (0, 0), None, scale, scale)
                else:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (imgArray[0][0].shape[1], imgArray[0][0].shape[0]),
                                                None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y] = cv2.cvtColor(imgArray[x][y], cv2.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank] * rows
        hor_con = [imageBlank] * rows
        for x in range(0, rows):
            hor[x] = np.hstack(imgArray[x])
        ver = np.vstack(hor)
    else:
        for x in range(0, rows):
            if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                imgArray[x] = cv2.resize(imgArray[x], (0, 0), None, scale, scale)
            else:
                imgArray[x] = cv2.resize(imgArray[x], (imgArray[0].shape[1], imgArray[0].shape[0]), None, scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x] = cv2.cvtColor(imgArray[x], cv2.COLOR_GRAY2BGR)
        hor = np.hstack(imgArray)
        ver = hor
    return ver


In [3]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
import os



cap = cv2.VideoCapture('C:/Users/hhhh/Desktop/haarcascades/highway.mp4')
#gemerate tracker object for tracking porpose
detection=[]
y1 = 500

h_lim = 80
w_lim = 80
offset = 6
count= 0

object1 = cv2.bgsegm.createBackgroundSubtractorMOG()

kernel = np.ones((5,5))

def calc_center(x,y,w,h):
    x1 = int(w/2)
    y1 = int(h/2)
    cx = x+x1
    cy =y+y1
    return cx,cy


while True :
    _,frame = cap.read()
    height, width, _ = frame.shape #(720, 1280, 3
    gray_img = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    buller_img = cv2.GaussianBlur(gray_img,(5,5),5)
    #apply back greound segmentaion
    seg_img = object1.apply(buller_img)
    dilate_img = cv2.dilate(seg_img,kernel) 
    # dilate_img = cv2.morphologyEx(seg_img,cv2.MORPH_OPEN,kernel)
    #find contour
    contours ,h = cv2.findContours(dilate_img,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    cv2.line(frame,(90,y1),(1200,y1),(0,0,255),2)
    for contour in contours :
        x,y,w,h =cv2.boundingRect(contour)
        if not (h>=h_lim and w>=w_lim ):
            continue
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
        center =calc_center(x,y,w,h)
        detection.append(center)
        cv2.circle(frame, center, 4, (0, 0, 255), -1)
        for (x,y)in detection :
            if (y<(y1+offset)) and (y>(y1-offset)) :
                count+=1
                cv2.line(frame,(90,y1),(1200,y1),(255,0,255),2)
                detection.remove((x,y))
        cv2.putText(frame, "Car Count : "+str(count), (320, 70),cv2.FONT_HERSHEY_COMPLEX, 2, (0, 0, 255), 4)  
    imgStack = stackImages(0.3,([frame],
                            [dilate_img]))
    cv2.imshow("Stack", imgStack)
 
    key = cv2.waitKey(10)
    if key == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()   

# motion detection and trackeing using oop


In [273]:
from abc import ABC, abstractmethod
import cv2

detection=[]
y1 = 500

h_lim = 100
w_lim = 100
offset = 6
count= 0
kernel = np.ones((5,5))




class backGroundSegmentaion:
    def __init__(self):
        pass
    
    def createObject(self):
        seg_object = cv2.bgsegm.createBackgroundSubtractorMOG()
        self.status()
        return seg_object
        
    def applyBackgroundSeg(self,filteredImage):
            object_seg = self.createObject()
            seg_image =object_seg.apply(filteredImage)
            return seg_image
    def status(self):
        print('Segmentaion Object Created Succesfuly')

            
class main_data(ABC):
    def __init__(self,path=''):
        self.path = path
        
    def load_images(self) :
        all_images=[]
        for image in os.listdir(self.path):
            image = cv2.imread(os.path.join(self.path,image))
            all_images.append(image)
        return all_images
    @abstractmethod
    def status():
        pass
    
    


class ImagePreprocessing(main_data):

    def __init__(self,seg_object,fillterKernel=None,kernel=None):
        self.fillterKernel =None
        self.kernel =None
        self.seg_object = seg_object

    def convertToGray(self, image):
        return cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    
    def filterImage (self,image):
        return cv2.GaussianBlur(image,self.fillterKernel)
    
    def dilate_image(self, image):
        dilated_image = cv2.dilate(image, self.kernel)
        return dilated_image
    
    def run_preprocessing(self,image):
        gray_img = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        buller_img = cv2.GaussianBlur(gray_img,self.fillterKernel,5)
        seg_img = self.seg_object.apply(buller_img)
        dilate_img = cv2.dilate(seg_img,kernel,iterations=5) 
        # self.status()
        return dilate_img
    
    def status(self) :
        print('image preprocessing done Successfuly ')
        
class RunCamera()  :
    

    def __init__(self):
            pass
        
    def getCountours(self,thresh):
            contours ,hist = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
            countour_lenth = len(contours)
            return contours ,countour_lenth

    def draw_Countours(self,image,cnt):
           return cv2.drawContours(image,cnt,-1,(255,0,0),3)
        
    @staticmethod 
    def calc_center(x,y,w,h):
        x1 = int(w/2)
        y1 = int(h/2)
        cx = x+x1
        cy =y+y1
        return cx,cy
    
    @staticmethod
    def stackImages(scale, imgArray):
            rows = len(imgArray)
            cols = len(imgArray[0])
            rowsAvailable = isinstance(imgArray[0], list)
            width = imgArray[0][0].shape[1]
            height = imgArray[0][0].shape[0]
            if rowsAvailable:
                for x in range(0, rows):
                    for y in range(0, cols):
                        if imgArray[x][y].shape[:2] == imgArray[0][0].shape[:2]:
                            imgArray[x][y] = cv2.resize(imgArray[x][y], (0, 0), None, scale, scale)
                        else:
                            imgArray[x][y] = cv2.resize(imgArray[x][y], (imgArray[0][0].shape[1], imgArray[0][0].shape[0]),
                                                        None, scale, scale)
                        if len(imgArray[x][y].shape) == 2: imgArray[x][y] = cv2.cvtColor(imgArray[x][y], cv2.COLOR_GRAY2BGR)
                imageBlank = np.zeros((height, width, 3), np.uint8)
                hor = [imageBlank] * rows
                hor_con = [imageBlank] * rows
                for x in range(0, rows):
                    hor[x] = np.hstack(imgArray[x])
                ver = np.vstack(hor)
            else:
                for x in range(0, rows):
                    if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                        imgArray[x] = cv2.resize(imgArray[x], (0, 0), None, scale, scale)
                    else:
                        imgArray[x] = cv2.resize(imgArray[x], (imgArray[0].shape[1], imgArray[0].shape[0]), None, scale, scale)
                    if len(imgArray[x].shape) == 2: imgArray[x] = cv2.cvtColor(imgArray[x], cv2.COLOR_GRAY2BGR)
                hor = np.hstack(imgArray)
                ver = hor
            return ver   


    def startrApp(self,image):
            global count
            cv2.line(frame,(90,y1),(1200,y1),(0,0,255),2)
            contours ,_ =self.getCountours(image)
            for contour in contours :
                x,y,w,h =cv2.boundingRect(contour)
                if not  (h>=h_lim and w>=w_lim):
                    continue
                cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
                centers =self.calc_center(x,y,w,h)
                detection.append(centers)
                cv2.circle(frame, centers, 4, (0, 0, 255), -1)
                #start detection
                for (x,y)in detection :
                    if (y<=(y1+offset)) and (y>=(y1-offset)) :
                        count+=1
                        cv2.line(frame,(90,y1),(1200,y1),(255,0,255),2)
                        detection.remove((x,y))
                
            cv2.putText(frame, "Car Count : "+str(count), (320, 70),cv2.FONT_HERSHEY_COMPLEX, 2, (0, 0, 255), 4) 
            imgStack = stackImages(0.3,([frame],
                            [dilate_img]))
            cv2.imshow("Stack", imgStack)

      
                    
    def status(self):
        print('Program Stoped ......')

In [274]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
import os



cap = cv2.VideoCapture('C:/Users/hhhh/Desktop/haarcascades/highway.mp4')
kernel = np.ones((5,5))

#create segmentaion object
seg=backGroundSegmentaion().createObject()

while True :
    _,frame = cap.read()
    height, width, _ = frame.shape 
    pre =ImagePreprocessing(seg,(5,5),kernel)
    dilate_img = pre.run_preprocessing(frame)
    video =RunCamera()
    video.startrApp(dilate_img)
    imgStack = stackImages(0.3,([frame],
                            [dilate_img]))
    cv2.imshow("Stack", imgStack)
 
    key = cv2.waitKey(10)
    if key == ord('q'):
        video.status()
        break
cap.release()
cv2.destroyAllWindows() 

Segmentaion Object Created Succesfuly
Program Stoped ......
